In [1]:
##### Functionnal version with optionnal mask padding and dropouts, see Transformer_V1.ipynb for example
import sys
path = '/gpfs/commons/groups/gursoy_lab/mstoll/'
sys.path.append(path)
from torch.utils.data import DataLoader
from torchviz import make_dot
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import LambdaLR, LinearLR, SequentialLR
from codes.models.utils import clear_last_line, print_file, number_tests, Unbuffered, plot_infos, plot_ini_infos

from torch.utils.tensorboard import SummaryWriter
from codes.models.data_form.DataForm import DataTransfo_1SNP, PatientList, Patient
import seaborn as sns

2024-02-22 15:54:46.245055: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 15:54:46.245120: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 15:54:46.247114: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-22 15:54:57.680399: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [265]:
##### Functionnal version with optionnal mask padding and dropouts, see Transformer_V1.ipynb for example
import sys
path = '/gpfs/commons/groups/gursoy_lab/mstoll/'
sys.path.append(path)


### imports
import torch
import torch.nn as nn
import numpy as np
import time
from torch.nn import functional as F
from sklearn.metrics import f1_score, accuracy_score

from codes.models.Transformers.Embedding import EmbeddingPheno
from codes.models.metrics import calculate_roc_auc, calculate_classification_report, calculate_loss, get_proba

### Transformer's instance
# B, S, E, H, HN, MH = Batch_len, Sentence_len, Embedding_len, Head_size, Head number, MultiHead size.

def custom_softmax(input_tensor, dim=-1):
    # Calculer le softmax classique
    softmax_output = F.softmax(input_tensor, dim)

    # Trouver les colonnes avec que des -inf
    nan_columns = torch.all(input_tensor == float('-inf'), dim=dim)

    # Remplacer les valeurs de softmax par 0 pour les colonnes avec que des -inf
    softmax_output[nan_columns] = 0

    return softmax_output

class TransformerGeneModel_V2(nn.Module):
    def __init__(self, pheno_method, Embedding, Head_size, binary_classes, n_head, n_layer, mask_padding=False, padding_token=None, p_dropout=0, device='cpu', loss_version='cross_entropy', gamma=2, alpha=1, instance_size=None, proj_embed=True, L1=False):
        super().__init__()
       
        self.Embedding_size = Embedding.Embedding_size
        self.binary_classes = binary_classes
        self.instance_size=instance_size
        self.proj_embed = proj_embed
        if not self.proj_embed:
            self.instance_size = self.Embedding_size
        if self.proj_embed:
            self.projection_embed = nn.Linear(self.Embedding_size, self.instance_size)
        self.classes_nb = 2 if self.binary_classes else 3
        self.blocks =PadMaskSequential(*[Block(self.instance_size, n_head=n_head, Head_size=Head_size, p_dropout=p_dropout) for _ in range(n_layer)]) #Block(self.instance_size, n_head=n_head, Head_size=Head_size) 
        self.ln_f = nn.LayerNorm(self.instance_size) # final layer norm
        self.lm_head_logits = nn.Linear(self.instance_size, self.classes_nb) 
        self.lm_head_proba = nn.Linear(self.instance_size,1) # plus one for the probabilities
        self.Embedding = Embedding
        self.mask_padding = mask_padding
        self.padding_token = padding_token
        self.padding_mask = None
        self.device = device
        self.pheno_method = pheno_method
        
        
        self.loss_version = loss_version
        self.gamma = gamma
        self.alpha = alpha
        self.shap = False
        self.L1 = L1
       
        self.diseases_embedding_table = Embedding.distinct_diseases_embeddings
        if self.pheno_method == 'Paul':
            self.counts_embedding_table = Embedding.counts_embeddings

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
            

    def create_padding_mask(self, diseases_sentence):
        B, S = np.shape(diseases_sentence)[0], np.shape(diseases_sentence)[1]
        mask = torch.where(diseases_sentence==self.padding_token)
        padding_mask_mat = torch.ones((B, S, S), dtype=torch.int)
        padding_mask_mat[mask] = 0
        padding_mask_mat.transpose(-2,-1)[mask] = 0

        
        padding_mask_probas = torch.zeros((B, S))
        padding_mask_probas[mask] = -torch.inf
        padding_mask_probas = padding_mask_probas.view(B, S)
        return padding_mask_mat.to(self.device), padding_mask_probas.to(self.device) # 1 if masked, 0 else

    def set_padding_mask_transformer(self, padding_mask, padding_mask_probas):
        self.padding_mask = padding_mask
        self.padding_mask_probas = padding_mask_probas
    

        


    def forward(self, diseases_sentence, counts_diseases, targets=None):

        diseases_sentence = diseases_sentence.to(torch.long)
        counts_diseases = counts_diseases.to(torch.long)

        Batch_len, Sentence_len = diseases_sentence.shape

        diseases_sentence = diseases_sentence.to(self.device)
        counts_diseases = counts_diseases.to(self.device)
        
        if targets!=None:
            targets = targets.to(self.device)

        if self.mask_padding:
            padding_mask, padding_mask_probas = self.create_padding_mask(diseases_sentence)
            self.set_padding_mask_transformer(padding_mask, padding_mask_probas)
            #self.blocks.set_padding_mask_sequential(self.padding_mask)

        diseases_sentences_embedded = self.diseases_embedding_table(diseases_sentence) # shape B, S, E

        x = diseases_sentences_embedded 
    
        if self.pheno_method == 'Paul':
            counts_diseases_embedded = self.counts_embedding_table(counts_diseases) # shape B, S, E
            x = x + counts_diseases_embedded # shape B, S, E 
        
        if self.proj_embed:
            x = self.projection_embed(x)
        x = self.blocks(x, self.padding_mask) # shape B, S, E
        x = self.ln_f(x) # shape B, S, E
        logits = self.lm_head_logits(x) #shape B, S, Classes_Numb, token logits
        weights_logits = self.lm_head_proba(x).view(Batch_len, Sentence_len)
        if self.mask_padding:
            weights_logits = weights_logits + self.padding_mask_probas
        probas = F.softmax(weights_logits) # shape B, S(represent the probas to be chosen)
        #if self.mask_padding:
           # probas = probas * self.padding_mask_probas
        
        logits = (logits.transpose(1, 2) @ probas.view(Batch_len, Sentence_len, 1)).view(Batch_len, self.classes_nb)# (B,Classes_Numb) Weighted Average logits
        loss = calculate_loss(logits, probas, targets, self.loss_version, self.gamma, self.alpha, L1=self.L1)

        if self.shap:
            return logits[:,0].view(Batch_len, 1)
        
        return logits, loss
    
    def forward_decomposed(self, diseases_sentence, diseases_count, targets=None):
        Batch_len, Sentence_len = diseases_sentence.shape
        diseases_sentence = diseases_sentence.to(self.device)
        counts_diseases = diseases_count.to(self.device)

        if self.mask_padding:
            padding_mask, padding_mask_probas = self.create_padding_mask(diseases_sentence)
            self.set_padding_mask_transformer(padding_mask, padding_mask_probas)
            #self.blocks.set_padding_mask_sequential(self.padding_mask)

        diseases_sentences_embedded = self.diseases_embedding_table(diseases_sentence) # shape B, S, E

        x = diseases_sentences_embedded 
        if self.pheno_method == 'Paul':
            counts_diseases_embedded = self.counts_embedding_table(counts_diseases) # shape B, S, E
            x = x + counts_diseases_embedded # shape B, S, E 
        if self.proj_embed:
            x = self.projection_embed(x)
        x= self.blocks.forward_decompose(x, self.padding_mask) # shape B, S, E
        x_out = self.ln_f(x) # shape B, S, E
        logits = self.lm_head_logits(x_out) #shape B, S, Classes_Numb, token logits
        weights_logits = self.lm_head_proba(x).view(Batch_len, Sentence_len)
        if self.mask_padding:
            weights_logits = weights_logits + self.padding_mask_probas
        probas = F.softmax(weights_logits) # shape B, S(represent the probas to be chosen)
        #if self.mask_padding:
           # probas = probas * self.padding_mask_probas
        
        logits = (logits.transpose(1, 2) @ probas.view(Batch_len, Sentence_len, 1)).view(Batch_len, self.classes_nb)# (B,Classes_Numb) Weighted Average logits
        loss = calculate_loss(logits, probas, targets, self.loss_version, self.gamma, self.alpha, L1=self.L1)

        return logits, probas, x_out, loss
    

    def predict(self, diseases_sentence, diseases_count):
        logits, _ = self(diseases_sentence, diseases_count) # shape B, Classes_nb
        return torch.argmax(logits, dim=1)  # (B,)
        
    def predict_proba(self, diseases_sentence, diseases_count):
        logits, _ = self(diseases_sentence, diseases_count)
        probabilities = F.softmax(logits, dim=1)
        return probabilities
    
    def evaluate(self, dataloader_test):
        print('beginning inference evaluation')
        start_time_inference = time.time()
        predicted_labels_list = []
        predicted_probas_list = []
        true_labels_list = []

        total_loss = 0.
        self.eval()
        with torch.no_grad():
            for batch_sentences, batch_counts, batch_labels in dataloader_test:


                logits, loss = self(batch_sentences, batch_counts,  batch_labels)
                total_loss += loss.item()
                predicted_labels = self.predict(batch_sentences, batch_counts)
                predicted_labels_list.extend(predicted_labels.cpu().numpy())
                predicted_probas = self.predict_proba(batch_sentences, batch_counts)
                predicted_probas_list.extend(predicted_probas.cpu().numpy())
                true_labels_list.extend(batch_labels.cpu().numpy())
        f1 = f1_score(true_labels_list, predicted_labels_list, average='macro')
        accuracy = accuracy_score(true_labels_list, predicted_labels_list)
        auc_score = calculate_roc_auc(true_labels_list, np.array(predicted_probas_list)[:, 1], return_nan=True)
        proba_avg_zero, proba_avg_one = get_proba(true_labels_list, predicted_probas_list)
    
        self.train()
        print(f'end inference evaluation in {time.time() - start_time_inference}s')
        return f1, accuracy, auc_score, total_loss/len(dataloader_test), proba_avg_zero, proba_avg_one, predicted_probas_list, true_labels_list


    def write_embedding(self, writer):
        if self.proj_embed:
            embedding_tensor = self.projection_embed(self.diseases_embedding_table.weight).detach().cpu().numpy()
        else:
            embedding_tensor = self.diseases_embedding_table.weight.detach().cpu().numpy()

        writer.add_embedding(embedding_tensor, metadata=self.Embedding.metadata, metadata_header=["Name","Label"])


class PadMaskSequential(nn.Sequential):
    def __init__(self, *args):
        super(PadMaskSequential, self).__init__(*args)
        self.padding_mask = None

    def set_padding_mask_sequential(self, padding_mask):
        self.padding_mask = padding_mask

    def forward(self, x, padding_mask=None):
        for module in self:
            #module.set_padding_mask_block(self.padding_mask)
            x = module(x, padding_mask)
        return x
    
    def forward_decompose(self, x, padding_mask):
        for module in self:
            #module.set_padding_mask_block(padding_mask)
            x = module.forward_decompose(x, padding_mask)
        return x
    
class Block(nn.Module):
    def __init__(self, instance_size, n_head, Head_size, p_dropout):
        super().__init__()
        self.sa = MultiHeadSelfAttention(n_head, Head_size, instance_size, p_dropout)
        self.ffwd = FeedForward(instance_size, p_dropout)
        self.ln1 = nn.LayerNorm(instance_size)
        self.ln2 = nn.LayerNorm(instance_size)
        self.padding_mask = None

    def set_padding_mask_block(self, padding_mask):
        self.padding_mask = padding_mask

    def forward(self, x, padding_mask):
        #self.sa.set_padding_mask_sa(self.padding_mask)
        #x = self.ln1(x)
        x = x + self.sa(x, padding_mask=None)
        x = self.ln1(x)
        x = x + self.ffwd(x)
        x = self.ln2(x)
        return x
    
    def forward_decompose(self, x, padding_mask=None):
        #self.sa.set_padding_mask_sa(self.padding_mask)
        out_sa= self.sa.forward_decompose(x, padding_mask)
        x = out_sa + x
        x = x + self.ffwd(x)
        x = self.ln2(x)
        return x

class MultiHeadSelfAttention(nn.Module):
    def __init__(self, n_head, Head_size, instance_size, p_dropout):
        super().__init__()
        self.qkv_network = nn.Linear(instance_size, Head_size * 3, bias = False) #group the computing of the nn.Linear for q, k and v, shape 
        self.proj = nn.Linear(Head_size, instance_size)
        self.attention_dropout = nn.Dropout(p_dropout)
        self.resid_dropout = nn.Dropout(p_dropout)

        self.multi_head_size = Head_size // n_head
        self.flash = False
        self.n_head = n_head
        self.Head_size = Head_size
        self.padding_mask = None

    def set_padding_mask_sa(self, padding_mask):
        self.padding_mask = padding_mask
    

        #self.dropout = nn.Dropout(dropout)
    def forward(self, x, padding_mask=None):
        # x of size (B, S, E)
        Batch_len, Sentence_len, _ = x.shape
        q, k, v  = self.qkv_network(x).split(self.Head_size, dim=2) #q, k, v of shape (B, S, H)
        # add a dimension to compute the different attention heads separately
        q_multi_head = q.view(Batch_len, Sentence_len, self.n_head, self.multi_head_size).transpose(1, 2) #shape B, HN, S, MH
        k_multi_head = k.view(Batch_len, Sentence_len, self.n_head, self.multi_head_size).transpose(1, 2)
        v_multi_head = v.view(Batch_len, Sentence_len, self.n_head, self.multi_head_size).transpose(1, 2)

        if self.flash:
            # efficient attention using Flash Attention CUDA kernels
            out = torch.nn.functional.scaled_dot_product_attention(q_multi_head, k_multi_head, v_multi_head, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)
        else:    
            attention_weights = (q_multi_head @ k_multi_head.transpose(-2, -1))/np.sqrt(self.multi_head_size) # shape B, S, S
            ### padding mask #####
            if padding_mask != None:
                padding_mask_weights = -(1-padding_mask)*(10**10)
                attention_weights = (attention_weights.transpose(0, 1)+padding_mask_weights).transpose(0, 1)
            #print(f'wei0={attention_weights}')
            attention_probas = F.softmax(attention_weights, dim=-1) # shape B, S, S
            if padding_mask != None:
                attention_probas = (attention_probas.transpose(0, 1)*padding_mask).transpose(0, 1)
           # attention_probas[attention_probas.isnan()]=0
            attention_probas = self.attention_dropout(attention_probas)

            #print(f'wei1={attention_probas}')
            #attention_probas = self.dropout(attention_probas)
            ## weighted aggregation of the values
            out = attention_probas @ v_multi_head # shape B, S, S @ B, S, MH = B, S, MH
        out = out.transpose(1, 2).contiguous().view(Batch_len, Sentence_len, self.Head_size)
        out = self.proj(out)
        out = self.resid_dropout(out)
        return out        
    
    def forward_decompose(self, x, padding_mask=None):
        # x of size (B, S, E)
        Batch_len, Sentence_len, _ = x.shape
        q, k, v  = self.qkv_network(x).split(self.Head_size, dim=2) #q, k, v of shape (B, S, H)
        # add a dimension to compute the different attention heads separately
        q_multi_head = q.view(Batch_len, Sentence_len, self.n_head, self.multi_head_size).transpose(1, 2) #shape B, HN, S, MH
        k_multi_head = k.view(Batch_len, Sentence_len, self.n_head, self.multi_head_size).transpose(1, 2)
        v_multi_head = v.view(Batch_len, Sentence_len, self.n_head, self.multi_head_size).transpose(1, 2)

        if self.flash:
            # efficient attention using Flash Attention CUDA kernels
            out = torch.nn.functional.scaled_dot_product_attention(q_multi_head, k_multi_head, v_multi_head, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)
        else:    
            attention_weights = (q_multi_head @ k_multi_head.transpose(-2, -1))/np.sqrt(self.multi_head_size) # shape B, S, S
            ### padding mask #####
            if padding_mask != None:
                padding_mask_weights = -(1-padding_mask)*(10**10)
                attention_weights = (attention_weights.transpose(0, 1)+padding_mask_weights).transpose(0, 1)


            self.attention_probas = F.softmax(attention_weights, dim=-1)
        
            if padding_mask != None:
                self.attention_probas = (self.attention_probas.transpose(0, 1)* padding_mask).transpose(0, 1)
            self.attention_probas.retain_grad()

           # attention_probas[attention_probas.isnan()]=0
            attention_probas_dropout = self.attention_dropout(self.attention_probas)
            


            #print(f'wei1={attention_probas}')
            #attention_probas = self.dropout(attention_probas)
            ## weighted aggregation of the values
            out = attention_probas_dropout @ v_multi_head # shape B, S, S @ B, S, MH = B, S, MH
        out = out.transpose(1, 2).contiguous().view(Batch_len, Sentence_len, self.Head_size)
        out = self.proj(out)
        #out = self.resid_dropout(out)
        return out

class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity"""
    def __init__(self, instance_size, p_dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear( instance_size, 4 * instance_size),
            nn.ReLU(),
            nn.Linear(4 * instance_size, instance_size),
            nn.Dropout(p_dropout)
        )

    def forward(self, x):
        return self.net(x)
        




In [266]:
import numpy as np
import torch
from sklearn.metrics import roc_auc_score, classification_report
from torch.nn import functional as F

def calculate_roc_auc(y_true, predicted_probabilities, return_nan=False, multi_class=None):
    # Check the number of unique classes
    num_classes = len(np.unique(y_true))
    print(f'num classes = {num_classes}')
    # Check if there is more than one class
    if num_classes > 1:
        if multi_class == None:
        # Compute ROC-AUC score
            roc_auc = roc_auc_score(y_true, predicted_probabilities)
        else:
            roc_auc = roc_auc_score(y_true, predicted_probabilities, multi_class=multi_class)

        return roc_auc
    else:
        # Return NaN if there is only one class
        if return_nan:
            return np.nan
        else:
            print("Only one class present in y_true. ROC AUC score is not defined in that case.")

def calculate_classification_report(true_labels_list, predicted_labels_list, return_nan=True):
    # Check the number of unique classes
    num_classes = len(np.unique(true_labels_list))

    # Check if there is more than one class
    if num_classes > 1:
        # Compute ROC-AUC score
        report = classification_report(true_labels_list, predicted_labels_list)
        return report
    else:
        # Return NaN if there is only one class
        if return_nan:
            return np.nan
        else:
            print("Only one class present in y_true. ROC AUC score is not defined in that case.")

def get_proba(true_labels_list, predicted_probas_list):
    avg_proba_zero = np.mean(np.array(predicted_probas_list)[:,0][np.array(true_labels_list)==0])
    avg_proba_one = np.mean(np.array(predicted_probas_list)[:,1][np.array(true_labels_list)==1])
    return avg_proba_zero, avg_proba_one

def calculate_loss(logits, logits_relevant, targets=None, loss_type='cross_entropy', gamma=None, alpha=None, L1=True):
    if targets is None:
            loss = None
    else:
        #target : shape B,
        
        if loss_type == 'cross_entropy':
            cross_entropy = F.cross_entropy(logits, targets)
            loss = cross_entropy
        elif loss_type == 'focal_loss':
            alphas = ((1 - targets) * (alpha-1)).to(torch.float) + 1
            probas = F.softmax(logits)
            certidude_factor =  (1-probas[[range(probas.shape[0]), targets]])**gamma * alphas
            cross_entropy = F.cross_entropy(logits, targets, reduce=False)
            loss = torch.dot(cross_entropy, certidude_factor)
        elif loss_type == 'predictions':
            probas = F.softmax(logits)
            predictions = (probas[:,1] > 0.5).to(int)
            loss = torch.sum((predictions-targets)**2)/len(predictions)
        
        if L1:
            loss_l1 = torch.norm(logits_relevant, p=1)
            return loss + loss_l1/10
        else:
            return loss

In [267]:
import os
import sys
sys.path.append('/gpfs/commons/groups/gursoy_lab/pmeddeb/phenotype_embedding')
import time
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

import torch
import torch.nn as nn

class SineCosineEncoding(nn.Module):
    def __init__(self, Embedding_size, max_len=1000):
        super(SineCosineEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, Embedding_size)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, Embedding_size, 2).float() * -(np.log(10000.0) / Embedding_size))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)

    def forward(self, x):

        return self.encoding.to(x.device)[x]

class ZeroEmbedding(nn.Module):
    def __init__(self, Embedding_size, max_len=1000):
        super(ZeroEmbedding, self).__init__()
        self.encoding = torch.zeros(max_len, Embedding_size)
       

    def forward(self, x):

        return self.encoding.to(x.device)[x]


class EmbeddingPheno(nn.Module):
    def __init__(self, method=None, counts_method=None, vocab_size=None, max_count_same_disease=None, Embedding_size=None, rollup_depth=4, freeze_embed=False, dicts=None):
        super(EmbeddingPheno, self).__init__()

        self.dicts = dicts
        self.rollup_depth = rollup_depth
        self.nb_distinct_diseases_patient = vocab_size
        self.Embedding_size = Embedding_size
        self.max_count_same_disease = None
        self.metadata = None
        self.counts_method = counts_method

        if self.dicts != None:
            id_dict = self.dicts['id']
            name_dict = self.dicts['name']
            cat_dict = self.dicts['cat']
            codes = list(id_dict.keys())
            diseases_present = self.dicts['diseases_present']
            self.metadata = [[name_dict[code], cat_dict[code]] for code in codes]

        
        if method == None:
            self.distinct_diseases_embeddings = nn.Embedding(vocab_size, Embedding_size)
            self.counts_embeddings = nn.Embedding(max_count_same_disease, Embedding_size)
            torch.nn.init.normal_(self.distinct_diseases_embeddings.weight, mean=0.0, std=0.02)
            torch.nn.init.normal_(self.counts_embeddings.weight, mean=0.0, std=0.02)

        elif method == 'Abby':
            embedding_file_diseases = f'/gpfs/commons/datasets/controlled/ukbb-gursoylab/mstoll/Data_Files/Embeddings/Abby/embedding_abby_no_1_diseases.pth'
            pretrained_weights_diseases = torch.load(embedding_file_diseases)[diseases_present]
            self.Embedding_size = pretrained_weights_diseases.shape[1]

            self.distinct_diseases_embeddings = nn.Embedding.from_pretrained(pretrained_weights_diseases, freeze=freeze_embed)
            self.counts_embeddings = nn.Embedding(max_count_same_disease, self.Embedding_size)



        elif method=='Paul':
            embedding_file_diseases = f'/gpfs/commons/datasets/controlled/ukbb-gursoylab/mstoll/Data_Files/Embeddings/Paul_Glove/glove_UKBB_omop_rollup_closest_depth_{self.rollup_depth}_no_1_diseases.pth'
            pretrained_weights_diseases = torch.load(embedding_file_diseases)[diseases_present]
            self.Embedding_size = pretrained_weights_diseases.shape[1]

            self.distinct_diseases_embeddings = nn.Embedding.from_pretrained(pretrained_weights_diseases, freeze=freeze_embed)
            if self.counts_method == 'SineCosine':
                self.counts_embeddings = SineCosineEncoding(self.Embedding_size, max_count_same_disease)
            elif self.counts_method == 'no_counts':
                self.counts_embeddings = ZeroEmbedding(self.Embedding_size, max_count_same_disease )
            else:

                self.counts_embeddings = nn.Embedding(max_count_same_disease, self.Embedding_size)
    def write_embedding(self, writer):
            embedding_tensor = self.distinct_diseases_embeddings.weight.data.detach().cpu().numpy()
            writer.add_embedding(embedding_tensor, metadata=self.metadata, metadata_header=["Name","Label"])


class EmbeddingPhenoCat(nn.Module):
    def __init__(self, pheno_method=None,  method=None, proj_embed=None, counts_method=None, Embedding_size=10, instance_size=10, rollup_depth=4, freeze_embed=False, dic_embedding_cat_params={}, dicts=None, device='cpu'):
        super(EmbeddingPhenoCat, self).__init__()

        self.rollup_depth = rollup_depth
        self.Embedding_size = Embedding_size
        self.max_count_same_disease = None
        self.dic_embedding_cat_params = dic_embedding_cat_params
        dic_embedding_cat = {}
        self.method = method
        self.pheno_method = pheno_method
        self.dicts = dicts
        self.proj_embed = proj_embed
        self.projection_embed = None
        self.instance_size = instance_size
        self.counts_method = counts_method

        self.device = device
        if self.dicts != None:
            id_dict = self.dicts['id']
            name_dict = self.dicts['name']
            cat_dict = self.dicts['cat']
            codes = list(id_dict.keys())
            diseases_present = self.dicts['diseases_present']
            self.metadata = [[name_dict[code], cat_dict[code]] for code in codes]

        for cat, max_number  in self.dic_embedding_cat_params.items():
        
            if cat=='diseases':
                if self.method == None:
                    dic_embedding_cat[cat] = nn.Embedding(max_number, Embedding_size)
                    torch.nn.init.normal_(dic_embedding_cat[cat].weight, mean=0.0, std=0.02)

                elif self.method == 'Abby':
                    embedding_file_diseases = f'/gpfs/commons/datasets/controlled/ukbb-gursoylab/mstoll/Data_Files/Embeddings/Abby/embedding_abby_no_1_diseases.pth'
                    pretrained_weights_diseases = torch.load(embedding_file_diseases)[diseases_present]
                    self.Embedding_size = pretrained_weights_diseases.shape[1]
                    dic_embedding_cat[cat] = nn.Embedding.from_pretrained(pretrained_weights_diseases, freeze=freeze_embed).to(self.device)

            

                elif self.method=='Paul':
                    embedding_file_diseases = f'/gpfs/commons/datasets/controlled/ukbb-gursoylab/mstoll/Data_Files/Embeddings/Paul_Glove/glove_UKBB_omop_rollup_closest_depth_{self.rollup_depth}_no_1_diseases.pth'
                    pretrained_weights_diseases = torch.load(embedding_file_diseases)[diseases_present]
                    self.Embedding_size = pretrained_weights_diseases.shape[1]
                    dic_embedding_cat[cat] = nn.Embedding.from_pretrained(pretrained_weights_diseases, freeze=freeze_embed).to(self.device)
                    
            elif cat == 'counts':
                if self.pheno_method == 'Paul':
                    if self.counts_method[cat] == 'SineCosine':
                        dic_embedding_cat[cat] = SineCosineEncoding(self.instance_size, max_number).to(self.device)
                    elif self.counts_method[cat] == 'no_counts':
                        dic_embedding_cat[cat] = ZeroEmbedding(self.instance_size, max_number).to(self.device)
                    else:
                        dic_embedding_cat[cat] = nn.Embedding(max_number, self.instance_size).to(self.device)
                        torch.nn.init.normal_(dic_embedding_cat[cat].weight, mean=0.0, std=0.02)

            elif cat == 'age':
                if self.counts_method[cat] == 'SineCosine':
                    dic_embedding_cat[cat] = SineCosineEncoding(self.instance_size, max_number).to(self.device)
                elif self.counts_method[cat] == 'no_counts':
                    dic_embedding_cat[cat] = ZeroEmbedding(self.instance_size, max_number).to(self.device)
                else:
                    dic_embedding_cat[cat] = nn.Embedding(max_number, self.instance_size).to(self.device)
                    torch.nn.init.normal_(dic_embedding_cat[cat].weight, mean=0.0, std=0.02)

                    

            else:
                dic_embedding_cat[cat] = nn.Embedding(max_number, self.instance_size).to(self.device)
                torch.nn.init.normal_(dic_embedding_cat[cat].weight, mean=0.0, std=0.02)

        if self.proj_embed:
            self.projection_embed = nn.Linear(self.Embedding_size, self.instance_size).to(self.device)

        self.dic_embedding_cat = dic_embedding_cat

    def forward(self, input_dict):
        list_env_embedded = []
        for key, value in input_dict.items():
            
            batch_len = len(value)

            if key=='diseases':
                diseases_sentences_embedded = self.dic_embedding_cat[key](value)
                if self.proj_embed:
                    diseases_sentences_embedded = self.projection_embed(diseases_sentences_embedded)

            elif key=='counts':
                if self.pheno_method == 'Paul':
                    counts_sentence_embedded = self.dic_embedding_cat[key](value)
                    diseases_sentences_embedded = diseases_sentences_embedded + counts_sentence_embedded
            

            else:
                list_env_embedded.append(self.dic_embedding_cat[key](value).view(batch_len, 1, self.instance_size))

        env_embedded = torch.concat(list_env_embedded, dim=1)

        return torch.concat([diseases_sentences_embedded, env_embedded], dim=1)
            



In [268]:
### creation of the reference model
#### framework constants:
model_type = 'transformer'
model_version = 'transformer_V2'
test_name = 'baseline_model'
pheno_method = 'Abby' # Paul, Abby
tryout = True # True if we are doing a tryout, False otherwise 
### data constants:
CHR = 1
SNP = 'rs673604'
rollup_depth = 4
binary_classes = False #nb of classes related to an SNP (here 0 or 1)
vocab_size = None # to be defined with data
padding_token = 0
prop_train_test = 0.8
load_data = True
save_data = False
remove_none = True
compute_features = False
indices=None
padding = True
list_env_features = ['age', 'sex']
### data format
batch_size = 200
data_share = 1#402555
seuil_diseases = 600
equalize_label = True
decorelate = False
threshold_corr = 0.9
threshold_rare = 1000
remove_rare = 'all' # None, 'all', 'one_class'
##### model constants
embedding_method = 'Abby' #None, Paul, Abby
counts_method = 'normal'#{'counts': 'SineCos', 'age':'SineCos'}
freeze_embedding = True
Embedding_size = 10 # Size of embedding.
proj_embed = True
instance_size = 10
n_head = 2# number of SA heads
n_layer = 1# number of blocks in parallel
Head_size = 8 # size of the "single Attention head", which is the sum of the size of all multi Attention heads
eval_epochs_interval = 5 # number of epoch between each evaluation print of the model (no impact on results)
eval_batch_interval = 40
p_dropout = 0.3 # proba of dropouts in the model
masking_padding = True # do we include padding masking or not
device = 'cuda' if torch.cuda.is_available() else 'cpu'
loss_version = 'cross_entropy' #cross_entropy or focal_loss
gamma = 2
alpha = 1
L1 = True
##### training constants
total_epochs = 50 # number of epochs
learning_rate_max = 0.001 # maximum learning rate (at the end of the warmup phase)
learning_rate_ini = 0.00001 # initial learning rate 
learning_rate_final = 0.0001
warm_up_frac = 0.5 # fraction of the size of the warmup stage with regards to the total number of epochs.
start_factor_lr = learning_rate_ini / learning_rate_max
end_factor_lr = learning_rate_final / learning_rate_max

warm_up_size = int(warm_up_frac*total_epochs)


In [269]:
dataT = DataTransfo_1SNP(SNP=SNP,
                         CHR=CHR,
                         method=pheno_method,
                         padding=padding,  
                         binary_classes=binary_classes,
                         pad_token=padding_token, 
                         load_data=load_data, 
                         save_data=save_data, 
                         compute_features=compute_features,
                         data_share=data_share,
                         prop_train_test=prop_train_test,
                         remove_none=remove_none,
                         rollup_depth=rollup_depth,
                         equalize_label=equalize_label,
                         seuil_diseases=seuil_diseases,
                         decorelate=decorelate,
                         threshold_corr=threshold_corr,
                         threshold_rare=threshold_rare,
                         remove_rare=remove_rare,
                         list_env_features=list_env_features, 
                         indices=indices)

In [270]:
patient_list = dataT.get_patientlist()

loading data
data_loaded in 0.2398667335510254 s
removing None values
padding data


In [271]:
test_model = True

In [272]:
if not test_model:   
    path = '/gpfs/commons/groups/gursoy_lab/mstoll/codes/'
    #check test name
    model_dir = path + f'logs/runs/SNPS/{str(CHR)}/{SNP}/{model_type}/{model_version}/{pheno_method}'
    model_plot_dir = path + f'logs/plots/tests/SNP/{str(CHR)}/{SNP}/{model_type}/{model_version}/{pheno_method}/'

    os.makedirs(model_dir, exist_ok=True)
    os.makedirs(model_plot_dir, exist_ok=True)
    #check number tests
    test_dir = f'{model_dir}/{test_name}/'
    print(test_dir)
    log_data_dir = f'{test_dir}/data/'
    log_tensorboard_dir = f'{test_dir}/tensorboard/'
    log_slurm_outputs_dir = f'{test_dir}/Slurm/Outputs/'
    log_slurm_errors_dir = f'{test_dir}/Slurm/Errors/'
    os.makedirs(log_data_dir, exist_ok=True)
    os.makedirs(log_tensorboard_dir, exist_ok=True)
    os.makedirs(log_slurm_outputs_dir, exist_ok=True)
    os.makedirs(log_slurm_errors_dir, exist_ok=True)


    log_data_path_pickle = f'{test_dir}/data/{test_name}.pkl'
    log_tensorboard_path = f'{test_dir}/tensorboard/{test_name}'
    log_slurm_outputs_path = f'{test_dir}/Slurm/Outputs/{test_name}.txt'
    log_slurm_error_path = f'{test_dir}/Slurm/Errors/{test_name}.txt'
    model_plot_path = path + f'logs/plots/tests/SNP/{str(CHR)}/{SNP}/{model_type}/{model_version}/{pheno_method}/{test_name}.png'

    sys.stdrerr = log_slurm_error_path

In [273]:
indices_train, indices_test = dataT.get_indices_train_test(nb_data=len(patient_list),prop_train_test=prop_train_test)
patient_list_transformer_train, patient_list_transformer_test = patient_list.get_transformer_data(indices_train.astype(int), indices_test.astype(int))
#creation of torch Datasets:
dataloader_train = DataLoader(patient_list_transformer_train, batch_size=batch_size, shuffle=True)
dataloader_test = DataLoader(patient_list_transformer_test, batch_size=batch_size, shuffle=True)


if patient_list.nb_distinct_diseases_tot==None:
    vocab_size = patient_list.get_nb_distinct_diseases_tot()
if patient_list.nb_max_counts_same_disease==None:
    max_count_same_disease = patient_list.get_max_count_same_disease()
max_count_same_disease = patient_list.nb_max_counts_same_disease
vocab_size = patient_list.get_nb_distinct_diseases_tot()

print(f'\n vocab_size : {vocab_size}, max_count : {max_count_same_disease}\n', 
    f'length_patient = {patient_list.get_nb_max_distinct_diseases_patient()}\n',
    f'sparcity = {patient_list.sparsity}\n',
    f'nombres patients  = {len(patient_list)}')

if not test_model:
    writer = SummaryWriter(log_tensorboard_path)





 vocab_size : 1440, max_count : 2
 length_patient = 119
 sparcity = 0.9138499474185424
 nombres patients  = 12348


In [274]:
Embedding  = EmbeddingPheno(method=embedding_method, 
                    counts_method=counts_method,
                    vocab_size=vocab_size, 
                    max_count_same_disease=max_count_same_disease, 
                    Embedding_size=Embedding_size, 
                    rollup_depth=rollup_depth, 
                    freeze_embed=freeze_embedding,
                    dicts=dataT.dicts)
### creation of the model
model = TransformerGeneModel_V2(pheno_method = pheno_method,
                            Embedding = Embedding,
                            Head_size=Head_size,
                            binary_classes=binary_classes,
                            n_head=n_head,
                            n_layer=n_layer,
                            mask_padding=masking_padding, 
                            padding_token=0, 
                            p_dropout=p_dropout, 
                            loss_version = loss_version, 
                            gamma = gamma,
                            alpha = alpha,
                            device = device,
                            proj_embed=proj_embed,
                            instance_size=instance_size)

# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')


0.18715 M parameters


In [275]:




                    
model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')


optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate_max)
lr_scheduler_warm_up = LinearLR(optimizer, start_factor=start_factor_lr , end_factor=1, total_iters=warm_up_size, verbose=False) # to schedule a modification in the learning rate
lr_scheduler_final = LinearLR(optimizer, start_factor=1, total_iters=total_epochs-warm_up_size, end_factor=end_factor_lr)
lr_scheduler = SequentialLR(optimizer, schedulers=[lr_scheduler_warm_up, lr_scheduler_final], milestones=[warm_up_size])

if not test_model:
    output_file = log_slurm_outputs_path
## Open tensor board writer
dic_features_list = {
'list_training_loss' : [],
'list_validation_loss' : [],
'list_proba_avg_zero' : [],
'list_proba_avg_one' : [],
'list_auc_validation' : [],
'list_accuracy_validation' : [],
'list_f1_validation' : [],
'epochs' : [] }

0.18715 M parameters


In [276]:
batch_diseases, batch_counts, batch_labels = next(iter(dataloader_test))

In [277]:
model.train()

TransformerGeneModel_V2(
  (projection_embed): Linear(in_features=128, out_features=10, bias=True)
  (blocks): PadMaskSequential(
    (0): Block(
      (sa): MultiHeadSelfAttention(
        (qkv_network): Linear(in_features=10, out_features=24, bias=False)
        (proj): Linear(in_features=8, out_features=10, bias=True)
        (attention_dropout): Dropout(p=0.3, inplace=False)
        (resid_dropout): Dropout(p=0.3, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=10, out_features=40, bias=True)
          (1): ReLU()
          (2): Linear(in_features=40, out_features=10, bias=True)
          (3): Dropout(p=0.3, inplace=False)
        )
      )
      (ln1): LayerNorm((10,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((10,), eps=1e-05, elementwise_affine=True)
    )
  )
  (ln_f): LayerNorm((10,), eps=1e-05, elementwise_affine=True)
  (lm_head_logits): Linear(in_features=10, out_features=3, bias=True)
  (lm_hea

In [278]:
logits, loss = model.forward(batch_diseases, batch_counts, batch_labels)

/scratch/ipykernel_21736/3967551285.py:132: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probas = F.softmax(weights_logits) # shape B, S(represent the probas to be chosen)


In [279]:
logits, loss, x_out, loss = model.forward_decomposed(batch_diseases, batch_counts, batch_labels)


/scratch/ipykernel_21736/3967551285.py:168: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probas = F.softmax(weights_logits) # shape B, S(represent the probas to be chosen)


In [249]:
model.blocks[0].sa.attention_probas

AttributeError: 'MultiHeadSelfAttention' object has no attribute 'attention_probas'

In [233]:
model.blocks[0].sa.attention_probas.shape

torch.Size([200, 2, 122, 122])

In [228]:
loss.backward()

In [161]:
logits, loss, x_out, loss = model.forward_decomposed(batch_diseases, batch_counts, batch_labels)

/scratch/ipykernel_21736/64605731.py:169: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probas = F.softmax(weights_logits) # shape B, S(represent the probas to be chosen)


In [165]:
loss

tensor(1.0707, grad_fn=<NllLossBackward0>)

In [166]:
model.blocks[0].sa.attention_probas

tensor([[[[0.5000, 0.5000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.5000, 0.5000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.5000, 0.5000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.5000, 0.5000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]],


        [[[0.0909, 0.0909, 0.0909,  ..., 0.0000, 0.0000, 0.0000],
          [0.0909, 0.0909, 0.0909,  ..., 0.0000, 0.0000, 0.0000],
          [0.0909, 0.0909, 0.0909,  ...,

In [140]:
loss.backward()

In [108]:
loss.grad

/scratch/ipykernel_21736/3355736690.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  loss.grad


In [43]:
def get_gpu_attributes(obj, prefix=''):
    """
    Fonction récursive pour obtenir les noms des attributs stockés sur GPU.
    """
    gpu_attributes = []
    if isinstance(obj, torch.Tensor) and obj.is_cuda:
        gpu_attributes.append(prefix)
    elif isinstance(obj, (list, tuple)):
        for i, item in enumerate(obj):
            gpu_attributes.extend(get_gpu_attributes(item, prefix=f"{prefix}[{i}]"))
    elif isinstance(obj, dict):
        for key, value in obj.items():
            gpu_attributes.extend(get_gpu_attributes(value, prefix=f"{prefix}.{key}"))
    elif hasattr(obj, '__dict__'):
        for key, value in obj.__dict__.items():
            gpu_attributes.extend(get_gpu_attributes(value, prefix=f"{prefix}.{key}"))
    return gpu_attributes

# Exemple d'objet avec des attributs stockés sur CPU et GPU
class ExempleObjet:
    def __init__(self):
        self.cpu_tensor = torch.randn(3, 3)
        self.gpu_tensor = torch.randn(3, 3).cuda()

   


In [44]:
model.to('cpu')

In [47]:
model.padding_mask = model.padding_mask.to('cpu')
model.padding_mask_probas = model.padding_mask.to('cpu')


In [48]:
get_gpu_attributes(model)

In [95]:
model.list_attention_layers[0][0][0][0]

In [96]:
model.padding_mask

In [ ]:
for name, attr in model.__dict__.items():
    if torch.is_tensor(attr) and attr.is_cuda:
        print(f"L'attribut {name} est sur le GPU.")

In [ ]:
model.to('cpu')

In [ ]:
# Training Loop
start_time_training = time.time()
print_file(output_file, f'Beginning of the program for {total_epochs} epochs', new_line=True)
# Training Loop
plot_ini_infos(model, output_file, dataloader_test, dataloader_train, writer, dic_features_list)
for epoch in range(1, total_epochs+1):

    start_time_epoch = time.time()
    total_loss = 0.0  
    
    #with tqdm(total=len(dataloader_train), position=0, leave=True) as pbar:
    for k, (batch_sentences, batch_counts, batch_labels) in enumerate(dataloader_train):
        
        batch_sentences = batch_sentences.to(device)
        batch_counts = batch_counts.to(device)
        batch_labels = batch_labels.to(device)

        # evaluate the loss
        logits, loss = model(batch_sentences, batch_counts, batch_labels)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
    

        total_loss += loss.item()

        optimizer.step()

        if k % eval_batch_interval == 0:
            clear_last_line(output_file)
            print_file(output_file, f'Progress in epoch {epoch}  = {round(k / len(dataloader_train)*100, 2)} %, time batch : {time.time() - start_time_epoch}', new_line=False)

    if epoch % eval_epochs_interval == 0:
        dic_features = plot_infos(model, output_file, epoch, total_loss, start_time_epoch, dataloader_train, dataloader_test, optimizer, writer, dic_features_list, model_plot_path)

    
    
    lr_scheduler.step()

dic_features = dic_features
model.to('cpu')
Embedding.to('cpu')
model.write_embedding(writer)
# Print time
print_file(output_file, f"Training finished: {int(time.time() - start_time_training)} seconds", new_line=True)
start_time = time.time()

with open(log_data_path_pickle, 'wb') as file:
    pickle.dump(model, file)
print('Model saved to %s' % log_data_path_pickle)


  ## Add hyper parameters to tensorboard
hyperparams = {"CHR" : CHR, "SNP" : SNP, "ROLLUP LEVEL" : rollup_depth,
            'PHENO_METHOD': pheno_method, 'EMBEDDING_METHOD': embedding_method,
            'EMBEDDING SIZE' : Embedding_size, 'ATTENTION HEADS' : n_head, 'BLOCKS' : n_layer,
            'LR':1 , 'DROPOUT' : p_dropout, 'NUM_EPOCHS' : total_epochs, 
            'BATCH_SIZE' : batch_size, 
            'PADDING_MASKING': masking_padding,
            'VERSION' : model_version,
            'NB_Patients'  : len(patient_list),
            'LOSS_VERSION'  : loss_version,
            }
writer.add_hparams(hyperparams, dic_features)

trained = True


In [ ]:
embeddings = nn.Embedding(10, 5)


In [ ]:
phenos = torch.arange(10)
weights = torch.arange(5)
embeddings(phenos)*weights